In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./master.csv')

## Checking missing values

In [3]:
for column_name in df.columns:
    num_value_missing =df[column_name].isnull().sum() 
    if num_value_missing>0:
        print(column_name,':', num_value_missing)

HDI for year : 19456


In [4]:
import copy
countryYr_HDI_dict = {}
for country, yr, HDI in df[['country','year','HDI for year']].values:
    try:
        countryYr_HDI_dict[country][int(yr)] = HDI
    except:
        countryYr_HDI_dict[country]={}
        countryYr_HDI_dict[country][int(yr)] = HDI

countryYr_HDI_dict_copy = copy.deepcopy(countryYr_HDI_dict)
for country in countryYr_HDI_dict.keys():
    temp = [[yr,-1 if str(HDI)=='nan' else HDI] for yr, HDI in countryYr_HDI_dict[country].items()]
    temp.sort(key=lambda x:x[0])
    temp = np.array(temp)
    arg_nonEmpty = np.argwhere(temp[:,1]>0)
    if len(arg_nonEmpty)==0:
        continue
    temp[:arg_nonEmpty[0].item(),1] =temp[arg_nonEmpty.min().item(),1] 
    temp[arg_nonEmpty[-1].item():,1] =temp[arg_nonEmpty.max().item(),1]
    for i,j in zip(arg_nonEmpty[:-1],arg_nonEmpty[1:]):
        ii, jj = i.item(), j.item()
        temp[ii:jj,1]=np.linspace(temp[ii,1],temp[jj,1],num=(j-i).item()+1)[:-1]
    for yr, HDI in temp:
        countryYr_HDI_dict[country][int(yr)]=HDI


In [5]:
for i in range(len(df)):
    country = df['country'].iloc[i]
    year = df['year'].iloc[i]
    df.at[i,'HDI for year'] = countryYr_HDI_dict[country][year]
    
df = pd.concat([df, pd.Series(data=df['HDI for year'].isnull().astype('float'),name='MissingHDI')],axis=1)
df.fillna(0, inplace=True)

In [6]:
df.to_csv('./WHO_Suicide_MissingHDIFixed.csv')